In [2]:
# Cell 1: Imports and Setup

import os

import zipfile

import torch

import torch.nn as nn

import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models

from PIL import Image

import cv2

import numpy as np

In [3]:
# Cell 2: Unzip the dataset



extract_path = '/kaggle/input/fruits/fruits'


In [4]:
classes = ['FreshApple', 'FreshBanana', 'FreshGrape', 'FreshGuava', 'FreshJujube', 'FreshOrange', 'FreshPomegranate', 'FreshStrawberry', 

           'RottenApple', 'RottenBanana', 'RottenGrape', 'RottenGuava', 'RottenJujube', 'RottenOrange', 'RottenPomegranate', 'RottenStrawberry']


In [5]:
class FruitDataset(Dataset):

    def __init__(self, root_dir, transform=None):

        self.root_dir = root_dir

        self.transform = transform

        self.classes = os.listdir(root_dir)

        self.file_list = []

        for class_name in self.classes:

            class_path = os.path.join(root_dir, class_name)

            self.file_list.extend([(os.path.join(class_path, f), class_name) for f in os.listdir(class_path)])



    def __len__(self):

        return len(self.file_list)



    def __getitem__(self, idx):

        img_path, class_name = self.file_list[idx]

        image = Image.open(img_path).convert('RGB')

        if self.transform:

            image = self.transform(image)

        label = classes.index(class_name)

        return image, label


In [6]:
# Cell 5: Define transforms and create dataset

transform = transforms.Compose([

    transforms.Resize(256),

    transforms.CenterCrop(224),

    transforms.ToTensor(),

    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

])



dataset = FruitDataset(extract_path, transform=transform)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)


In [7]:
# Cell 6: Load and modify pre-trained ResNet model

model = models.resnet50(pretrained=True)

num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, len(classes))

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 157MB/s] 


In [8]:
# Cell 7: Define loss function and optimizer

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
print(torch.cuda.is_available())  # Returns True if CUDA is available, False otherwise


True


In [10]:
# Cell 8: Train the model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.to(device)



num_epochs = 10

for epoch in range(num_epochs):

    model.train()

    running_loss = 0.0

    for inputs, labels in dataloader:

        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}')


Epoch 1/10, Loss: 0.4707135884911594
Epoch 2/10, Loss: 0.170788095226521
Epoch 3/10, Loss: 0.13976493171522383
Epoch 4/10, Loss: 0.09873821020806135
Epoch 5/10, Loss: 0.09023547910682862
Epoch 6/10, Loss: 0.0663026663193653
Epoch 7/10, Loss: 0.08383496877940316
Epoch 8/10, Loss: 0.03920772675725805
Epoch 9/10, Loss: 0.05357802617834118
Epoch 10/10, Loss: 0.03814680188675861


In [18]:
# Cell 9: Save the model

torch.save(model.state_dict(), 'fruit_freshness_model.pth')

In [14]:
# Cell 10: Define prediction function

def predict(image):

    img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    img_t = transform(img)

    batch_t = torch.unsqueeze(img_t, 0).to(device)



    model.eval()

    with torch.no_grad():

        out = model(batch_t)



    probabilities = torch.nn.functional.softmax(out[0], dim=0)

    predicted_class_index = probabilities.argmax().item()

    return classes[predicted_class_index], probabilities[predicted_class_index].item()


In [17]:
# Cell 11: Live video classification

cap = cv2.VideoCapture(2)



while True:

    ret, frame = cap.read()

    if not ret:

        break



    class_name, confidence = predict(frame)

    freshness = "Fresh" if "Fresh" in class_name else "Rotten"



    cv2.putText(frame, f"{class_name} ({freshness})", (10, 30),

                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.putText(frame, f"Confidence: {confidence:.2f}", (10, 60),

                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)



    cv2.imshow('Fruit Freshness Classifier', frame)



    if cv2.waitKey(1) & 0xFF == ord('q'):

        break



cap.release()

cv2.destroyAllWindows()

[ WARN:0@1401.515] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@1401.515] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


error: OpenCV(4.10.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
